# Edge

> TODO fill in description

Essentially just a wrapper class for `asyncio.Queue`. Will eventually have additional features, such as logging and debugging.

In [1]:
#|default_exp edge

In [2]:
#|hide
from nbdev.showdoc import *

In [3]:
#|hide
import nbdev; nbdev.nbdev_export()

In [4]:
#|export
import asyncio
from collections import deque
from typing import Type

from fbdev import Packet
from fbdev.packet_container import DequePacketContainer
from fbdev.utils import StateHandler, StateView, EventHandler

In [5]:
#|export
__all__ = ['Edge']

In [6]:
#|hide
show_doc(fbdev.Edge)

NameError: name 'fbdev' is not defined

In [15]:
#|export
class Edge:
    def __init__(self,
                 input_node,
                 output_node,
                 maxsize:int):
        self.id = None  # TODO: implement id
        self._input_node = input_node
        self._output_node = output_node
        self.enabled = StateHandler(True)
        
        self._empty = StateHandler(True)
        self._full = StateHandler(False)
        self.empty = StateView(self._empty)
        self.full = StateView(self._full)
        self.has_changed = EventHandler()
        
        self.maxsize = maxsize
        self._packets = deque(maxlen=maxsize)
        
    def _update_status_events(self):        
        if len(self._packets) > 0: self._empty.set(False)
        else: self._empty.set(True)
        if len(self._packets) == self.maxsize and self.maxsize is not None: self._full.set(True)
        else: self._full.set(False)
    
    def load(self, packet: Packet):
        if self._full.get(): raise Exception("Port is full.")
        self._packets.append(packet)
        self._update_status_events()
        self.has_changed._trigger()
        
    def unload(self):
        if self._empty.get(): raise Exception("Port is empty.")
        packet = self._packets.popleft()
        self._update_status_events()
        self.has_changed._trigger()
        return packet
        
    def size(self):
        return len(self._packets)
    
    def _load_left(self, packet: Packet):
        if self._full.get(): raise Exception("Port is full.")
        self._packets.appendleft(packet)
        self._update_status_events()
        self.has_changed._trigger()
    
    def _unload_right(self):
        if self._empty.get(): raise Exception("Port is empty.")
        packet = self._packets.pop()
        self._update_status_events()
        self.has_changed._trigger()
        return packet
        
    def _unload_all(self):
        if self._empty.get(): raise Exception("Port is empty.")
        packets = list(self._packets)
        self._packet.clear()
        self._update_status_events()
        self.has_changed._trigger()
        return packet

In [19]:
edge = Edge(None, None, maxsize=3)

# Load data1
changed_event1 = edge.has_changed.subscribe()
edge.load(Packet("data1"))
assert changed_event1.is_set()

# Load data2
edge.load(Packet("data2"))

# Load data3
edge.load(Packet("data3"))
assert edge.full.get()

# Unload data1
assert edge.unload().get_data() == 'data1'
assert edge.size() == 2

# Unload data3
assert edge._unload_right().get_data() == 'data3'
assert edge.size() == 1

# Unload data2
assert edge.unload().get_data() == 'data2'
assert edge.size() == 0